# Quantitative Value Investing Strategy

"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

We start by importing the required packages.

In [288]:
import numpy as np #The Numpy numerical computing library
import pandas as pd #The Pandas data science library
import requests #The requests library for HTTP requests in Python
import xlsxwriter #The XlsxWriter libarary for 
import math #The Python math module
from scipy import stats #The SciPy stats module
from statistics import mean #To calculate the arithmetic mean

We will now read our data into the pandas data frame.

In [318]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Now we know from working on this data set in previous projects that the data set contains four stocks that have been delisted and four stocks with None value in them. So we will go on to remove those rows of data from this list of stocks.

I later realised that there are some NaN value that have to be dealt with. Therefore, I am adding a line of code for the same. Don't run it now, when you find the error, then remove the '#' and run all later cells.

In [319]:
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC', 'VIAC', 'WLTW'])] #to get rid of the 4 rows with delisted stocks
stocks = stocks[~stocks['Ticker'].isin(['CTL', 'ETFC', 'MYL', 'NBL'])] #to get rid of rows with None value in them
#stocks = stocks[~stocks['Ticker'].isin(['ALXN', 'AON', 'BRK.B', 'CERN', 'DISCK', 'FLIR', 'FOX', 'FRT', 'GOOG', 'MXIM', 'NWS', 'PBCT', 'TROW', 'UA', 'XLNX'])]
                           #to deal with an error that arises later on
stocks # we don't care about the index here as we will create a new data frame and store the values there

,Ticker
0,A
1,AAL
2,AAP
3,AAPL
4,ABBV
...,...
500,YUM
501,ZBH
502,ZBRA
503,ZION


Next we will import our API token in order to make our API call.

In [56]:
from secrets import IEX_CLOUD_API_TOKEN

We can now proceed to testing our API call.

## Making our Test API Call:

We are well familiar with the process of API calling by now. But it is good practice to make a single API call to test run the code and its performance. We can then reuse the same code to make our batch API calls. 
Now it's time to structure our API calls to IEX cloud. We need the following information from the API:
- One year stock returns
We use the same method as in the pervious projects to create the required API URL. We quickly find that the 'peRatio' attribute in the /quote/ endpoint is the one we need.

In [5]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url)
data.status_code

200

And we get a response code of 200. Everything is working perfectly fine. We now parse our API call for the required metric.

In [6]:
data = data.json()
data

{'avgTotalVolume': 74242461,
 'calculationPrice': 'close',
 'change': 2.34,
 'changePercent': 0.01564,
 'close': 0,
 'closeSource': 'ioilffac',
 'closeTime': None,
 'companyName': 'Apple Inc',
 'currency': 'USD',
 'delayedPrice': None,
 'delayedPriceTime': None,
 'extendedChange': None,
 'extendedChangePercent': None,
 'extendedPrice': None,
 'extendedPriceTime': None,
 'high': 0,
 'highSource': None,
 'highTime': None,
 'iexAskPrice': 0,
 'iexAskSize': 0,
 'iexBidPrice': 0,
 'iexBidSize': 0,
 'iexClose': 157.66,
 'iexCloseTime': 1694764087963,
 'iexLastUpdated': 1703715414317,
 'iexMarketPercent': 0.00190555111569882,
 'iexOpen': 155.61,
 'iexOpenTime': 1699335115139,
 'iexRealtimePrice': 156.56,
 'iexRealtimeSize': 51,
 'iexVolume': 1267,
 'lastTradeTime': 1682036957728,
 'latestPrice': 161.27,
 'latestSource': 'Close',
 'latestTime': 'July 21, 2022',
 'latestUpdate': 1698211385119,
 'latestVolume': None,
 'low': 0,
 'lowSource': None,
 'lowTime': None,
 'marketCap': 2587053493119,
 

In [7]:
data['peRatio']

26.13

We can now proceed to making our batch API call.

## Executing A Batch API Call & Building Our DataFrame

We start by creating chunks of the data and adding them to a list.

In [320]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]
symbol_groups = list(chunks(stocks['Ticker'], 100))
print(len(symbol_groups))
symbol_groups

5


[0         A
 1       AAL
 2       AAP
 3      AAPL
 4      ABBV
       ...  
 95     CINF
 96       CL
 97      CLX
 98      CMA
 99    CMCSA
 Name: Ticker, Length: 100, dtype: object,
 100     CME
 101     CMG
 102     CMI
 103     CMS
 104     CNC
        ... 
 198      GE
 199    GILD
 200     GIS
 201      GL
 202     GLW
 Name: Ticker, Length: 100, dtype: object,
 203       GM
 204     GOOG
 205    GOOGL
 206      GPC
 207      GPN
        ...  
 299     MCHP
 300      MCK
 301      MCO
 302     MDLZ
 303      MDT
 Name: Ticker, Length: 100, dtype: object,
 304     MET
 305     MGM
 306     MHK
 307     MKC
 308    MKTX
        ... 
 401     ROP
 402    ROST
 403     RSG
 404     RTX
 405    SBAC
 Name: Ticker, Length: 100, dtype: object,
 406    SBUX
 407    SCHW
 408     SEE
 409     SHW
 410    SIVB
        ... 
 500     YUM
 501     ZBH
 502    ZBRA
 503    ZION
 504     ZTS
 Name: Ticker, Length: 97, dtype: object]

After which we'll take each chunk join them using the comma as a seperator so that we can pass them onto IEX cloud within the limit of 100 stocks at a time for batch API calls.

In [321]:
symbol_strings = []
for i in range(0, len(symbol_groups)): # loop runs for each chunk
    symbol_strings.append(','.join(symbol_groups[i])) # join() joins all keys in the chunk and append() add each joined chunk
    print(symbol_strings[i])

A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,ADI,ADM,ADP,ADSK,AEE,AEP,AES,AFL,AIG,AIV,AIZ,AJG,AKAM,ALB,ALGN,ALK,ALL,ALLE,ALXN,AMAT,AMCR,AMD,AME,AMGN,AMP,AMT,AMZN,ANET,ANSS,ANTM,AON,AOS,APA,APD,APH,APTV,ARE,ATO,ATVI,AVB,AVGO,AVY,AWK,AXP,AZO,BA,BAC,BAX,BBY,BDX,BEN,BF.B,BIIB,BIO,BK,BKNG,BKR,BLK,BLL,BMY,BR,BRK.B,BSX,BWA,BXP,C,CAG,CAH,CARR,CAT,CB,CBOE,CBRE,CCI,CCL,CDNS,CDW,CE,CERN,CF,CFG,CHD,CHRW,CHTR,CI,CINF,CL,CLX,CMA,CMCSA
CME,CMG,CMI,CMS,CNC,CNP,COF,COG,COO,COP,COST,COTY,CPB,CPRT,CRM,CSCO,CSX,CTAS,CTSH,CTVA,CTXS,CVS,CVX,CXO,D,DAL,DD,DE,DFS,DG,DGX,DHI,DHR,DIS,DISCK,DISH,DLR,DLTR,DOV,DOW,DPZ,DRE,DRI,DTE,DUK,DVA,DVN,DXC,DXCM,EA,EBAY,ECL,ED,EFX,EIX,EL,EMN,EMR,EOG,EQIX,EQR,ES,ESS,ETN,ETR,EVRG,EW,EXC,EXPD,EXPE,EXR,F,FANG,FAST,FB,FBHS,FCX,FDX,FE,FFIV,FIS,FISV,FITB,FLIR,FLS,FLT,FMC,FOX,FOXA,FRC,FRT,FTI,FTNT,FTV,GD,GE,GILD,GIS,GL,GLW
GM,GOOG,GOOGL,GPC,GPN,GPS,GRMN,GS,GWW,HAL,HAS,HBAN,HBI,HCA,HD,HES,HIG,HII,HLT,HOLX,HON,HPE,HPQ,HRB,HRL,HSIC,HST,HSY,HUM,HWM,IBM,ICE,IDXX,IEX,IFF,ILMN,INCY,INFO,INTC,IN

We now create an empty data frame in pandas to push the data, from the API call, into it.

In [59]:
my_columns = ['Ticker', 'Price', 'Price-to-Earnings Ratio', 'Number of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)

Now, we'll push the data into our data frame. We will use both, stats and quote endpoint to get the data we require.

In [11]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
#     print(symbol_strings)
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
                                        pd.Series([symbol, 
                                                   data[symbol]['quote']['latestPrice'],
                                                   data[symbol]['quote']['peRatio'],
                                                   'N/A'
                                                   ], 
                                                  index = my_columns), 
                                        ignore_index = True)
        
    
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,A,129.93,31.35,N/A
1,AAL,14.33,-3.89,N/A
2,AAP,194.59,21.98,N/A
3,AAPL,157.57,26.39,N/A
4,ABBV,152.25,21.61,N/A
...,...,...,...,...
492,YUM,123.70,22.45,N/A
493,ZBH,108.83,110.81,N/A
494,ZBRA,335.46,22.33,N/A
495,ZION,54.54,5.67,N/A


## Removing Glamour Stocks

The opposite of a "value stock" is a "glamour stock".

Since the goal of this strategy is to identify the 50 best value stocks from our universe, our next step is to remove glamour stocks from the DataFrame. We also remove the stocks witha negative PE ratio as there is a large body of research that suggests that such stocks are not good value.

We'll sort the DataFrame by the stocks' price-to-earnings ratio, and drop all stocks outside the top 50.

In [12]:
final_dataframe.sort_values('Price-to-Earnings Ratio', inplace = True)
final_dataframe = final_dataframe[final_dataframe['Price-to-Earnings Ratio'] > 0] # we don't want negative values of PE ratio
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe.drop('index', axis=1, inplace = True)
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,GOOGL,118.28,1.05,N/A
1,NRG,36.83,2.3,N/A
2,EBAY,46.74,2.64,N/A
3,AMZN,129.81,3.06,N/A
4,SYF,33.25,4.58,N/A
5,PVH,63.16,4.6,N/A
6,COF,119.50,4.63,N/A
7,AIG,53.84,4.68,N/A
8,F,13.00,4.75,N/A
9,KSS,32.00,4.82,N/A


## Calculating the Number of Shares to Buy

We now need to calculate the number of shares we need to buy.

To do this, we will use the portfolio_input function that we created in our momentum project.

I have included this function below.

In [13]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input("Enter the value of your portfolio:")

    try:
        val = float(portfolio_size)
    except ValueError:
        print('Please enter the portfolio value as a number without currency')
        portfolio_size = input("Enter the value of your portfolio:")

Use the portfolio_input function to accept a portfolio_size variable from the user of this script.

In [14]:
portfolio_input()

Enter the value of your portfolio: 1000000


We can now use the global portfolio_size variable to calculate the number of shares that our strategy should purchase.

In [18]:
position_size = float(portfolio_size) / len(final_dataframe.index)
for i in range(0, len(final_dataframe['Ticker'])):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size / final_dataframe['Price'][i])
final_dataframe

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,GOOGL,118.28,1.05,169
1,NRG,36.83,2.3,543
2,EBAY,46.74,2.64,427
3,AMZN,129.81,3.06,154
4,SYF,33.25,4.58,601
5,PVH,63.16,4.6,316
6,COF,119.50,4.63,167
7,AIG,53.84,4.68,371
8,F,13.00,4.75,1538
9,KSS,32.00,4.82,625


Now that we have an overly simplistics model ready for our quantitative value investive strategy. We can move on to create a more realistic model which combines various different metrics into a Robust Value (RV) metric which is the used to sort the data and then calculate number of shares to buy.

## Building a More Realistic Value Strategy:

Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a composite basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
- Enterprise Value divided by Gross Profit (EV/GP)

Some of these metrics aren't provided directly by the IEX Cloud API, and must be computed after pulling raw data. Two ratios, i.e., EV/EBITDA and EV/GP, have to be calculated. Looking through the IEX Cloud Documentation page, we find that the /advanced-stats/ endpoint will gie us all the required metrics. So we'll use it along with the /quote/ endpoint to make the API call.
We'll start by testing the API call and then calculating each data point from scratch.

In [35]:
symbol = 'A'
api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol}&token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'A': {'advanced-stats': {'beta': 1.1021076549740556,
   'totalCash': 1208674472,
   'currentDebt': 1882951438,
   'revenue': 6697511674,
   'grossProfit': 3563603170,
   'totalRevenue': 6597433563,
   'EBITDA': 1861985461,
   'revenuePerShare': 22.21,
   'revenuePerEmployee': 498306.57,
   'debtToEquity': 2.041750826475379,
   'profitMargin': 0.2012025462288544,
   'enterpriseValue': 40945928884,
   'enterpriseValueToRevenue': 6.3,
   'priceToSales': 5.94,
   'priceToBook': 7.49,
   'forwardPERatio': 26.84481255324878,
   'pegRatio': 0.7918170325057805,
   'peHigh': 24.41917405129633,
   'peLow': 18.599835769928408,
   'week52highDate': '2021-08-24',
   'week52lowDate': '2022-06-16',
   'putCallRatio': 0.94959877255104,
   'companyName': 'Agilent Technologies Inc.',
   'marketcap': 39101383672,
   'week52high': 184.14,
   'week52low': 116.82,
   'week52highSplitAdjustOnly': 186.87,
   'week52highDateSplitAdjustOnly': '2021-08-31',
   'week52lowSplitAdjustOnly': 114.76,
   'week52lowDa

We'll test run all the mertics we need.

In [36]:
# P/E Ratio
pe_ratio = data[symbol]['quote']['peRatio']
print('The P/E ratio is',pe_ratio)
# P/B Ratio
pb_ratio = data[symbol]['advanced-stats']['priceToBook']
print('The P/B ratio is',pb_ratio)
#P/S Ratio
ps_ratio = data[symbol]['advanced-stats']['priceToSales']
print('The P/S ratio is',ps_ratio)
# EV/EBITDA
enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
ebitda = data[symbol]['advanced-stats']['EBITDA']
ev_to_ebitda = enterprise_value/ebitda
print('The EV/EBITDA ratio is',ev_to_ebitda)
# EV/GP
gross_profit = data[symbol]['advanced-stats']['grossProfit']
ev_to_gross_profit = enterprise_value/gross_profit
print('The EV/GP ratio is',ev_to_gross_profit)

The P/E ratio is 31.05
The P/B ratio is 7.49
The P/S ratio is 5.94
The EV/EBITDA ratio is 21.990466489469544
The EV/GP ratio is 11.490036047981178


Now let's move on to building our DataFrame. You'll notice that I use the abbreviation rv often. It stands for robust value, which is what we'll call this sophisticated strategy moving forward.

In [293]:
rv_columns = [
    'Ticker',
    'Price',
    'Price-to-Earnings Ratio',
    'PE Percentile',
    'Price-to-Book Ratio',
    'PB Percentile',
    'Price-to-Sales Ratio',
    'PS Percentile',
    'EV/EBITDA',
    'EV/EBITDA Percentile',
    'EV/GP',
    'EV/GP Percentile',
    'RV Score',
    'Number of Shares to Buy']

rv_dataframe = pd.DataFrame(columns = rv_columns)
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy


In [229]:
for symbol_string in symbol_strings:
    api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        #ev_to_ebitda = enterprise_value/ebitda
        ev_to_gross_profit = enterprise_value/gross_profit
        rv_dataframe = rv_dataframe.append(
        pd.Series(
        [symbol,
         data[symbol]['quote']['latestPrice'],
         data[symbol]['quote']['peRatio'],
         'N/A',
         data[symbol]['advanced-stats']['priceToBook'],
         'N/A',
         data[symbol]['advanced-stats']['priceToSales'],
         'N/A',
         ev_to_ebitda,
         'N/A',
         ev_to_gross_profit,
         'N/A',
         'N/A',
         'N/A'
        ], index = rv_columns),
        ignore_index = True)
rv_dataframe

TypeError: unsupported operand type(s) for /: 'NoneType' and 'NoneType'

We see a TypeError "unsupported operand type(s) for /: 'NoneType' and 'int'". This could be for various reasons, but, judging by the syntax it's probably beacuse there are None values in enterprise_value or ebitda variables. We also see the same error in the next line which indicates to me the enterprise_value variable is the problem. To deal with this we'll use a try except block.

In [275]:
for symbol_string in symbol_strings:
    api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        rv_dataframe = rv_dataframe.append(
        pd.Series(
        [symbol,
         data[symbol]['quote']['latestPrice'],
         data[symbol]['quote']['peRatio'],
         'N/A',
         data[symbol]['advanced-stats']['priceToBook'],
         'N/A',
         data[symbol]['advanced-stats']['priceToSales'],
         'N/A',
         ev_to_ebitda,
         'N/A',
         ev_to_gross_profit,
         'N/A',
         'N/A',
         'N/A'
        ], index = rv_columns),
        ignore_index = True)
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,A,128.55,30.78,N/A,7.44,N/A,5.83,N/A,22.635526,N/A,11.434002,N/A,N/A,N/A
1,AAL,14.20,-3.76,N/A,-1.06,N/A,0.27,N/A,-22.486916,N/A,0.985240,N/A,N/A,N/A
2,AAP,198.29,21.61,N/A,4.1,N/A,1.09,N/A,12.009411,N/A,2.590214,N/A,N/A,N/A
3,AAPL,159.26,25.5,N/A,38.63,N/A,6.81,N/A,19.873431,N/A,15.717714,N/A,N/A,N/A
4,ABBV,151.86,21.51,N/A,16.59,N/A,4.8,N/A,11.376746,N/A,8.060589,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,121.67,21.84,N/A,-4.21,N/A,5.27,N/A,20.157518,N/A,8.872391,N/A,N/A,N/A
493,ZBH,110.00,109.3,N/A,1.98,N/A,3.06,N/A,18.358168,N/A,5.377904,N/A,N/A,N/A
494,ZBRA,336.80,21.9,N/A,6,N/A,3.13,N/A,17.277513,N/A,7.086523,N/A,N/A,N/A
495,ZION,54.03,5.58,N/A,1.39,N/A,2.3,N/A,4.309299,N/A,2.162295,N/A,N/A,N/A


## Dealing With Missing Data in Our DataFrame:

Our DataFrame contains some missing data because all of the metrics we require are not available through the API we're using.

We will use pandas' isnull method to identify missing data:

In [276]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
28,ALXN,186.00,60.27,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
40,AON,280.46,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
71,BRK.B,298.98,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
88,CERN,98.42,48.89,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
134,DISCK,25.07,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
183,FLIR,59.58,32.9,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
187,FOX,33.38,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
190,FRT,106.77,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
201,GOOG,113.75,None,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A
321,MXIM,107.52,34.76,N/A,None,N/A,None,N/A,NaN,N/A,NaN,N/A,N/A,N/A


We have a lot of NaN values. It's probably because IEX Cloud does not provide data for these stocks.

We could deal with this by choosing to fill the NaN value with the mean non NaN value of that column. This will, however, skew the percentile as the percentile calucation is dependent on the length of the series. therefore instead I will simply remove these rows from the data set. We need the Ticker to remove these rows from the data.

In [277]:
a = []
for index in rv_dataframe[rv_dataframe.isnull().any(axis=1)].index:
    a.append(rv_dataframe['Ticker'][index])
print(a)

['ALXN', 'AON', 'BRK.B', 'CERN', 'DISCK', 'FLIR', 'FOX', 'FRT', 'GOOG', 'MXIM', 'NWS', 'PBCT', 'TROW', 'UA', 'XLNX']


So, we'll go back and remove these rows.

Re run the code to see the changes.

In [294]:
for symbol_string in symbol_strings:
    api_url = f'https://sandbox.iexapis.com/stable/stock/market/batch/?types=quote,advanced-stats&symbols={symbol_string}&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(api_url).json()
    for symbol in symbol_string.split(','):
        enterprise_value = data[symbol]['advanced-stats']['enterpriseValue']
        ebitda = data[symbol]['advanced-stats']['EBITDA']
        gross_profit = data[symbol]['advanced-stats']['grossProfit']
        try:
            ev_to_ebitda = enterprise_value/ebitda
        except TypeError:
            ev_to_ebitda = np.NaN
        try:
            ev_to_gross_profit = enterprise_value/gross_profit
        except TypeError:
            ev_to_gross_profit = np.NaN
        rv_dataframe = rv_dataframe.append(
        pd.Series(
        [symbol,
         data[symbol]['quote']['latestPrice'],
         data[symbol]['quote']['peRatio'],
         'N/A',
         data[symbol]['advanced-stats']['priceToBook'],
         'N/A',
         data[symbol]['advanced-stats']['priceToSales'],
         'N/A',
         ev_to_ebitda,
         'N/A',
         ev_to_gross_profit,
         'N/A',
         'N/A',
         'N/A'
        ], index = rv_columns),
        ignore_index = True)
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,A,126.16,31.33,N/A,7.65,N/A,5.870,N/A,22.638663,N/A,11.309351,N/A,N/A,N/A
1,AAL,14.20,-3.74,N/A,-1.06,N/A,0.264,N/A,-21.909059,N/A,0.993375,N/A,N/A,N/A
2,AAP,192.59,21.66,N/A,3.96,N/A,1.090,N/A,11.923457,N/A,2.539265,N/A,N/A,N/A
3,AAPL,161.73,25.39,N/A,37.85,N/A,6.730,N/A,19.892224,N/A,15.126820,N/A,N/A,N/A
4,ABBV,148.58,21.30,N/A,16.66,N/A,4.800,N/A,11.184259,N/A,8.128448,N/A,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,YUM,120.65,22.34,N/A,-4.19,N/A,5.360,N/A,20.678145,N/A,9.484616,N/A,N/A,N/A
478,ZBH,108.00,106.78,N/A,1.96,N/A,3.020,N/A,17.599019,N/A,5.288438,N/A,N/A,N/A
479,ZBRA,335.33,22.10,N/A,6.00,N/A,3.070,N/A,16.567147,N/A,7.158641,N/A,N/A,N/A
480,ZION,55.37,5.69,N/A,1.41,N/A,2.300,N/A,4.512857,N/A,2.266428,N/A,N/A,N/A


Now, if we run the statement from earlier to print rows that contain missing data, nothing should be returned:

In [285]:
rv_dataframe[rv_dataframe.isnull().any(axis=1)]

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy


## Calculating Value Percentiles:

We now need to calculate value score percentiles for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

- Price-to-earnings ratio
- Price-to-book ratio
- Price-to-sales ratio
- EV/EBITDA
- EV/GP
Here's how we'll do this:

In [295]:
metrics = {
            'Price-to-Earnings Ratio': 'PE Percentile',
            'Price-to-Book Ratio':'PB Percentile',
            'Price-to-Sales Ratio': 'PS Percentile',
            'EV/EBITDA':'EV/EBITDA Percentile',
            'EV/GP':'EV/GP Percentile'
}
for metric in metrics:
    for index in rv_dataframe.index:
        rv_dataframe.loc[index, metrics[metric]] = stats.percentileofscore(
                                                    rv_dataframe[metric], rv_dataframe.loc[index, metric])
        
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,A,126.16,31.33,73.8589,7.65,81.9502,5.870,82.0539,22.638663,83.8174,11.309351,80.2905,N/A,N/A
1,AAL,14.20,-3.74,3.3195,-1.06,6.22407,0.264,1.45228,-21.909059,0.829876,0.993375,4.14938,N/A,N/A
2,AAP,192.59,21.66,52.4896,3.96,60.6846,1.090,17.0124,11.923457,42.3237,2.539265,15.3527,N/A,N/A
3,AAPL,161.73,25.39,60.6846,37.85,96.888,6.730,84.6473,19.892224,75.5187,15.126820,90.4564,N/A,N/A
4,ABBV,148.58,21.30,51.1411,16.66,92.1162,4.800,74.6888,11.184259,37.5519,8.128448,64.1079,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,YUM,120.65,22.34,54.1494,-4.19,5.80913,5.360,78.8382,20.678145,78.0083,9.484616,73.444,N/A,N/A
478,ZBH,108.00,106.78,98.1328,1.96,30.1867,3.020,56.0166,17.599019,68.8797,5.288438,39.2116,N/A,N/A
479,ZBRA,335.33,22.10,53.9419,6.00,74.5851,3.070,56.4315,16.567147,63.9004,7.158641,56.4315,N/A,N/A
480,ZION,55.37,5.69,8.09129,1.41,18.5685,2.300,43.9834,4.512857,8.09129,2.266428,14.3154,N/A,N/A


## Calculating the RV Score:

We'll now calculate our RV Score (which stands for Robust Value), which is the value score that we'll use to filter for stocks in this investing strategy.

The RV Score will be the arithmetic mean of the 4 percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [296]:
for index in rv_dataframe.index:
    value_percentiles = []
    for metric in metrics.keys():
        value_percentiles.append(rv_dataframe.loc[index, metrics[metric]])
    rv_dataframe.loc[index, 'RV Score'] = mean(value_percentiles)
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,A,126.16,31.33,73.8589,7.65,81.9502,5.870,82.0539,22.638663,83.8174,11.309351,80.2905,80.3942,N/A
1,AAL,14.20,-3.74,3.3195,-1.06,6.22407,0.264,1.45228,-21.909059,0.829876,0.993375,4.14938,3.19502,N/A
2,AAP,192.59,21.66,52.4896,3.96,60.6846,1.090,17.0124,11.923457,42.3237,2.539265,15.3527,37.5726,N/A
3,AAPL,161.73,25.39,60.6846,37.85,96.888,6.730,84.6473,19.892224,75.5187,15.126820,90.4564,81.639,N/A
4,ABBV,148.58,21.30,51.1411,16.66,92.1162,4.800,74.6888,11.184259,37.5519,8.128448,64.1079,63.9212,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477,YUM,120.65,22.34,54.1494,-4.19,5.80913,5.360,78.8382,20.678145,78.0083,9.484616,73.444,58.0498,N/A
478,ZBH,108.00,106.78,98.1328,1.96,30.1867,3.020,56.0166,17.599019,68.8797,5.288438,39.2116,58.4855,N/A
479,ZBRA,335.33,22.10,53.9419,6.00,74.5851,3.070,56.4315,16.567147,63.9004,7.158641,56.4315,61.0581,N/A
480,ZION,55.37,5.69,8.09129,1.41,18.5685,2.300,43.9834,4.512857,8.09129,2.266428,14.3154,18.61,N/A


## Selecting the 50 Best Value Stocks:

As before, we can identify the 50 best value stocks in our universe by sorting the DataFrame on the RV Score column and dropping all but the top 50 entries.

In [297]:
rv_dataframe.sort_values(by = 'RV Score', ascending = False, inplace = True)
rv_dataframe = rv_dataframe[:50]
rv_dataframe.reset_index(drop = True, inplace = True)
rv_dataframe

,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,NOW,457.19,420.95,100,24.17,95.4357,15.33,97.7178,123.981463,99.7925,18.400618,95.0207,97.5934,N/A
1,PAYC,327.19,87.27,96.6805,20.01,94.3983,18.28,99.5851,54.075777,98.9627,19.365443,96.2656,97.1784,N/A
2,NVDA,181.82,47.68,90.4564,17.46,92.9461,15.76,98.3402,36.635485,97.0954,23.626277,98.5477,95.4772,N/A
3,CDNS,169.54,64.17,94.8133,16.88,92.3237,15.21,97.5104,49.942456,98.7552,15.793782,91.4938,94.9793,N/A
4,IDXX,404.52,46.43,88.7967,52.60,98.1328,10.25,92.7386,33.289001,95.0207,17.912508,94.3983,93.8174,N/A
5,AMT,267.20,47.03,89.6266,22.67,95.0207,12.20,95.8506,27.129850,91.0788,22.561570,97.5104,93.8174,N/A
6,LLY,329.36,51.19,92.7386,34.75,96.6805,10.86,93.361,32.238518,94.3983,15.386947,90.8714,93.61,N/A
7,ADSK,200.00,91.99,97.0954,66.10,98.9627,9.79,91.4938,57.968604,99.1701,10.747386,78.0083,92.9461,N/A
8,ABMD,291.70,95.73,97.5104,8.78,82.7801,13.07,96.2656,43.885275,98.3402,14.766926,89.6266,92.9046,N/A
9,MA,352.72,35.85,80.4979,49.58,97.9253,17.40,98.7552,29.026679,92.1162,17.413989,93.361,92.5311,N/A


## Calculating the Number of Shares to Buy:

We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [298]:
portfolio_input()

Enter the value of your portfolio: 1000000


In [302]:
position_size = float(portfolio_size) / len(rv_dataframe.index)
for index in rv_dataframe.index:
    rv_dataframe.loc[index, 'Number of Shares to Buy'] = math.floor(position_size / rv_dataframe['Price'][index])
rv_dataframe

c:\users\suer\appdata\local\programs\python\python36-32\lib\site-packages\pandas\core\indexing.py:1763: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,Ticker,Price,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score,Number of Shares to Buy
0,NOW,457.19,420.95,100,24.17,95.4357,15.33,97.7178,123.981463,99.7925,18.400618,95.0207,97.5934,43
1,PAYC,327.19,87.27,96.6805,20.01,94.3983,18.28,99.5851,54.075777,98.9627,19.365443,96.2656,97.1784,61
2,NVDA,181.82,47.68,90.4564,17.46,92.9461,15.76,98.3402,36.635485,97.0954,23.626277,98.5477,95.4772,109
3,CDNS,169.54,64.17,94.8133,16.88,92.3237,15.21,97.5104,49.942456,98.7552,15.793782,91.4938,94.9793,117
4,IDXX,404.52,46.43,88.7967,52.60,98.1328,10.25,92.7386,33.289001,95.0207,17.912508,94.3983,93.8174,49
5,AMT,267.20,47.03,89.6266,22.67,95.0207,12.20,95.8506,27.129850,91.0788,22.561570,97.5104,93.8174,74
6,LLY,329.36,51.19,92.7386,34.75,96.6805,10.86,93.361,32.238518,94.3983,15.386947,90.8714,93.61,60
7,ADSK,200.00,91.99,97.0954,66.10,98.9627,9.79,91.4938,57.968604,99.1701,10.747386,78.0083,92.9461,100
8,ABMD,291.70,95.73,97.5104,8.78,82.7801,13.07,96.2656,43.885275,98.3402,14.766926,89.6266,92.9046,68
9,MA,352.72,35.85,80.4979,49.58,97.9253,17.40,98.7552,29.026679,92.1162,17.413989,93.361,92.5311,56


## Formatting Our Excel Output:

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files. We have done this enough times now to guess how this works.

In [311]:
writer = pd.ExcelWriter('value_strategy.xlsx', engine='xlsxwriter')
rv_dataframe.to_excel(writer, sheet_name='Value Strategy', index = False)

## Creating the Formats We'll Need For Our .xlsx File:

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

- String format for tickers
- $XX.XX format for stock prices
- $XX,XXX format for market capitalization
- Integer format for the number of shares to purchase
- Decimal formats with 2 decimal for each valuation metric

Since we already built some formats in past sections of this course, I've included them below for you. Run this code cell before proceeding.

In [312]:
background_color = '#ffffff'
font_color = '#000000'
# let's start with string format
string_format = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
dollar_format = writer.book.add_format(
        {
            'num_format': '$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
integer_format = writer.book.add_format(
        {
            'num_format': '0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
float_format = writer.book.add_format(
        {
            'num_format': '0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )
bold_format = writer.book.add_format({'bold': True})
# this assigns formats to each of the variables

In [315]:
column_formats = {
                    'A': ['Ticker', string_format],
                    'B': ['Price', dollar_format],
                    'C': ['Price-to-Earnings Ratio', float_format],
                    'D': ['PE Percentile', float_format],
                    'E': ['Price-to-Book Ratio', float_format],
                    'F': ['PB Percentile',float_format],
                    'G': ['Price-to-Sales Ratio', float_format],
                    'H': ['PS Percentile', float_format],
                    'I': ['EV/EBITDA', float_format],
                    'J': ['EV/EBITDA Percentile', float_format],
                    'K': ['EV/GP', float_format],
                    'L': ['EV/GP Percentile', float_format],
                    'M': ['RV Score', float_format],
                    'N': ['Number of Shares to Buy', integer_format]
                 }

for column in column_formats.keys():
    writer.sheets['Value Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Value Strategy'].write(f'{column}1', column_formats[column][0], bold_format)

In [316]:
writer.save()

Voila! That's it for our Quantitative Value Strategy. We'll make an update to this soon.